In [112]:
import numpy as np
import torch
import torch.nn.functional as F  #激活函数
from torch import nn
from torch import optim  #优化器
from torchvision import transforms as tf
import matplotlib.pyplot as plt
import os
from PIL import Image

In [113]:
def get_data(root_dir):
  paths=[]
  labels=[]
  for root, dirs, files in os.walk(root_dir):
    for file in files:
      paths.append(os.path.join(root, file))
      index=file.find('_')
      labels.append(file[:index])
  return paths,labels

def get_folder(root_dir):
  folder_name=[]
  for entry in os.listdir(root_dir):
    full_path=os.path.join(root_dir, entry)
    if os.path.isdir(full_path):
      folder_name.append(entry)
  return folder_name

In [114]:
class CifarDataset(torch.utils.data.Dataset):
  def __init__(self, root_dir):
    #初始化数据集，包含一系列图片与对应标签
    super().__init__()
    #self.data_path=root_dir
    dataset=get_folder(root_dir)
    data, labels=get_data(root_dir)
    self.data=data
    self.label=labels
    if(dataset[0][-4:]=='test'):
      #test_labels=get_folder(dataset[0])
      test_data, test_labels=get_data(dataset[0])
      #train_labels=get_folder(dataset[1])
      train_data, train_labels=get_data(dataset[1])
    else:
      #test_labels=get_folder(dataset[1])
      test_data, test_labels=get_data(dataset[1])
      #train_labels=get_folder(dataset[0])
      train_data, train_labels=get_data(dataset[0])
    self.train_label=train_labels
    self.train=train_data
    self.test_label=test_labels
    self.test=test_data
    #raise NotImplementedError
  
  def __len__(self):
    #返回数据集大小
    return len(self.data)
    #raise NotImplementedError
  
  def __getitem__(self, index):
    #返回第index个样本
    return self.data[index], self.label[index]
    #raise NotImplementedError
  
  

In [115]:
def get_mean(dataset):
  expect_sum, var_sum=torch.zeros(3), torch.zeros(3)
  size=len(dataset)
  transform=tf.Compose([tf.ToTensor(), tf.Normalize((0,), (1,))])
  for i in dataset:
    image=Image.open(i[0])
    img_tensor=transform(image)
    expect_sum=expect_sum+torch.mean(img_tensor, dim=[1,2])
    var_sum=var_sum+torch.mean(img_tensor**2, dim=[1,2])
  mean=expect_sum/size
  std=var_sum/size
  return mean, std

In [116]:
#FCNModel
def get_dataloader(train):
  dataset=CifarDataset('..\cifar10')
  mean, std=get_mean(dataset)
  t=tf.Compose([tf.ToTensor(), tf.Normalize(mean, std)])
  i, j=0, 0
  for i in range(len(dataset.train)):
    dataset.train[i]=t(Image.open(dataset.train[i]))
  for j in range(len(dataset.test)):
    dataset.test[j]=t(Image.open(dataset.test[j]))
  if(train):
    dataset=list(zip(dataset.train, dataset.train_label))
  else:
    dataset=list(zip(dataset.test, dataset.test_label))
  
  batch_size=train_batch_size if train else test_batch_size
  dataloader=torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  #print(dataset.data.size())
  return dataloader

train_batch_size=128
test_batch_size=64


In [117]:
class FCNModel(torch.nn.Module):
  def __init__(self):
    super(FCNModel, self).__init__()
    self.fc1=torch.nn.Linear(32*32*1, 32)
    self.fc2=(32, 10)

  def forward(self, input_data):
    x=input_data.view(-1, 28*28*1)
    x=self.fc1(x)
    x=F.relu(x)
    x=self.fc2(x)
    out=F.log_softmax(x, dim=-1)
    return out

In [118]:
fcn_model=FCNModel()
optimizer=torch.optim.Adam(fcn_model.parameters(), lr=0.001)
train_loss_list=[]
train_count_list=[]
def train(epoch):
  fcn_model.train(True)
  train_dataloader=get_dataloader(train)
  print("start training")
  for id, (train_data, train_label) in enumerate(train_dataloader):
    optimizer.zero_grad()
    out=fcn_model(train_data)
    loss=F.nll_loss(out, train_label)
    loss.backward()
    optimizer.step()
    if id%200==0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,  id * len(train_data), len(train_dataloader.dataset),100. * id / len(train_dataloader), loss.item()))
      train_loss_list.append(loss.item())
      train_count_list.append(id*train_batch_size+(epoch-1)*len(train_dataloader))
  print('end training')

In [119]:
if __name__ =='__main__':
  for i in range(5):
    train(i)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# 保存一下模型和优化器
torch.save(fcn_model.state_dict(),"fcn_model.pt") #保存模型参数
torch.save(optimizer.state_dict(), 'optimizer.pt') #保存优化器参数
# state_dict变量存放训练过程中需要学习的权重，state_dict作为python的字典对象将每一层的参数映射成tensor张量

# 模型的加载
fcn_model.load_state_dict(torch.load("fcn_model.pt"))
optimizer.load_state_dict(torch.load("optimizer.pt"))

In [ ]:
def test():
    test_loss = 0
    correct = 0
    fcn_model.eval()  #设置为评估模式
    test_dataloader = get_dataloader(train=False)  #导入测试数据集
    with torch.no_grad():  #不需要计算梯度
        for data, label in test_dataloader:
            output = fcn_model(data)
            test_loss += F.nll_loss(output, label, reduction='sum').item()  
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置
            # 第一个1表示找第2维的最大值（如果是第1维的最大值则设为0）
            # output.data.max(2, keepdim=True)会返回一个数组，第一个是output数组中第2维度的最大值是多少，
            # 第二个是最大值的位置在哪里
            # 因此[1]表示此时只关心最大值是多少
            correct += pred.eq(label.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)  #计算平均损失
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [ ]:
#CNNModel
def get_dataloader(train):
  dataset=CifarDataset('..\cifar10')
  mean, std=get_mean(dataset)
  dataset=tf.Compose([tf.ToTensor(), tf.Normalize(mean, std)])
  batch_size=train_batch_size if train else test_batch_size
  dataloader=torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  return dataloader

class CNNModel(torch.nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)